# Advent of Code 2023 - Day 10: **Pipe Maze**

### Preparation

In [134]:
import collections 
from math import gcd
from functools import reduce
with open('input.txt', 'r') as f:
    data = f.read().splitlines()

In [135]:
low = 0
high = 1

class FlipFlop:
    def __init__(self, destinations):
        self.state = False
        self.destinations = destinations
        
    def __call__(self, signal, src):
        if signal == high:
            return []
        self.state = not self.state
        return [(dst, int(self.state)) for dst in self.destinations] # when was off -> return high
    
    def __repr__(self) -> str:
        return f'FF({self.state}, {self.destinations})'
        
class Conjunction:
    def __init__(self, destinations, sources):
        self.memory = {src : low for src in sources}
        self.destinations = destinations
        
    def __call__(self, signal, src):
        self.memory[src] = signal
        if low in self.memory.values(): # not all high
            return [(dst, high) for dst in self.destinations]
        else:
            return [(dst, low) for dst in self.destinations]
        
    def __repr__(self) -> str:
        return f'Conj({self.memory}, {self.destinations})'
        

In [136]:
modules = dict()
conjunctions = dict()
dest_to_src = collections.defaultdict(list) # to save src for conjunctions
broadcaster = None

rx_premod = ""

for line in data:
    line = line.split(' -> ')
    name = line[0][1:] if line[0] != 'broadcaster' else 'broadcaster'
    mod_type = line[0][0]
    destinations = line[1].split(', ')
    
    if name == 'broadcaster':
        broadcaster = destinations
    elif mod_type == '%':
        modules[name] = FlipFlop(destinations)
    elif mod_type == '&':
        conjunctions[name] = destinations
    
    for d in destinations:
        dest_to_src[d].append(name)
        
    
for name, destinations in conjunctions.items():
    modules[name] = Conjunction(destinations, dest_to_src[name])
    if 'rx' in destinations:
        rx_premod = name
    
# modules, conjunctions, dest_to_src, broadcaster, rx_premod

In [137]:
def push_button():
    queue = collections.deque()
    start = [('broadcaster', low, module) for module in broadcaster] # src, signal, dst
    queue.extend(start)

    counter = [1, 0]

    while queue:
        src, signal, dst = queue.popleft()
        counter[signal] += 1
        # print(src, f'-{signal}->', dst)
        
        if dst not in modules:
            continue
        
        for new_dst, new_signal in modules[dst](signal, src):
            queue.append((dst, new_signal, new_dst))
            
    return counter
        
        
counter = [0,0]
for i in range(1000):
    ret = push_button()   
    counter[0] += ret[0]
    counter[1] += ret[1]
    
counter, counter[0] * counter[1]

([16195, 42243], 684125385)

### Part 2

In [138]:
lengths = {}
visited = {
    name: 0 for name, module in modules.items() if rx_premod in module.destinations
}

count = 0
while True:
    count += 1
    queue = collections.deque(
        [('broadcaster', low, module) for module in broadcaster] # src, signal, dst
    )
    
    events = []

    # one button press
    while queue:
        src, signal, dst = queue.popleft()
        
        if dst not in modules:
            continue
        
        if dst == rx_premod and signal == high:
            visited[src] = 1
            
            if src not in lengths:
                lengths[src] = count
                
            if all(visited.values()):
                ret = reduce(lambda product, length: product * length // gcd(product, length), lengths.values(), 1)
                print(ret, lengths)
                break
        
        for new_dst, new_signal in modules[dst](signal, src):
            queue.append((dst, new_signal, new_dst))      
    else:
        continue
    break

answers:

    5347346877, too low
    
    22803499706691, too low
    225872806380073 -> yes